# Simulations for Figure 1
This file contains the gpytorch simulations for figure 1. To run this file, set the "gpu" variable below to true if you would like to use Cuda or false if you would not. Then, simply run each of the cells. It will output two csv files that the HS-SVD-fig1.Rmd file will use to create the figure.

In [1]:
gpu = True

In [2]:
import math
import torch
import gpytorch
import time
from matplotlib import pyplot as plt
import gc
import csv

import faiss
from matplotlib import pyplot as plt
import psutil
import numpy as np

from gpytorch.models import ApproximateGP
from gpytorch.variational.nearest_neighbor_variational_strategy import NNVariationalStrategy
# Make plots inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import psutil
def get_mem():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss



import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

import urllib.request
import os
import pandas as pd
from scipy.io import loadmat
from math import floor

import urllib.request
import os
from scipy.io import loadmat
from math import floor

import urllib.request
import os
import pandas as pd
from scipy.io import loadmat
from math import floor

max_vram = 0
max_ram = 0


max_vram = 0
def vram_usage():
    global max_vram
    max_vram = max(max_vram, torch.cuda.memory_allocated())


# Make plots inline
%matplotlib inline



## Loading Data

In [3]:
smoke_test = ('CI' in os.environ)


data_files_x = ['Data/train_x_20k.csv','Data/train_x_31k.csv','Data/train_x_50k.csv','Data/train_x_80k.csv','Data/train_x_120k.csv','Data/train_x_200k.csv']
data_files_y = ['Data/train_y_20k.csv','Data/train_y_31k.csv','Data/train_y_50k.csv','Data/train_y_80k.csv','Data/train_y_120k.csv','Data/train_y_200k.csv']


csvfile = pd.read_csv('Data/test_x_200k.csv', header = None)
test_x = torch.tensor(csvfile[0]).float()
csvfile = pd.read_csv('Data/test_y_200k.csv', header = None)
test_y = torch.tensor(csvfile[0]).float()


test_x = test_x.contiguous()
test_y = test_y.contiguous()



# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)

# SKI

In [ ]:
time_ski = []

for i in np.arange(0,6):
    csvfile = pd.read_csv(data_files_x[i], header = None)
    train_x = torch.tensor(csvfile[0]).float()
    train_x = torch.reshape(train_x,[-1,1])
    csvfile = pd.read_csv(data_files_y[i], header = None)
    train_y = torch.tensor(csvfile[0]).float()
    train_n = int(len(train_x))
    train_x = train_x.contiguous()
    train_y = train_y.contiguous()
    print(train_x.shape)
    print(train_y.shape)
    print(test_x.shape)
    print(test_y.shape)
    if gpu:
        train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

    class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

            # SKI requires a grid size hyperparameter. This util can help with that. Here we are using a grid that has the same number of points as the training data (a ratio of 1.0). Performance can be sensitive to this parameter, so you may want to adjust it for your own problem on a validation set.
            grid_size = gpytorch.utils.grid.choose_grid_size(train_x,1.0/25.0)

            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.GridInterpolationKernel(
                    gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=1), grid_size=grid_size, num_dims=1
                )
            )


        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



    my_batch_size = 320
    smoke_test = False

    from torch.utils.data import TensorDataset, DataLoader
    train_dataset = TensorDataset(train_x, train_y)#batch_size=1024
    train_loader = DataLoader(train_dataset, batch_size=my_batch_size, shuffle=True)

    test_dataset = TensorDataset(test_x, test_y)
    test_loader = DataLoader(test_dataset, batch_size=my_batch_size, shuffle=False)

    # this is for running the notebook in our testing framework
    import os
    smoke_test = ('CI' in os.environ)
    training_iterations = 32

    mem_begin = get_mem()

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPRegressionModel(train_x, train_y, likelihood)


    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    if gpu:
        mll = mll.cuda()

    begin = time.time()

    for i in tqdm.tqdm(range(training_iterations), desc="Train"):
        optimizer.zero_grad()
        max_vram = max(max_vram, torch.cuda.memory_allocated())
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        max_vram = max(max_vram, torch.cuda.memory_allocated())
        optimizer.step()

    uTime = time.time()-begin
    uRAM = (get_mem() - mem_begin)/(1024**2)
    uVRAM = max_vram / (1024 ** 2)
    # print(time.time()-begin)
    # print("RAM: ",(get_mem() - mem_begin)/(1024**2))
    # print("VRAM: ", max_vram / (1024 ** 2))

    # Get into evaluation (predictive posterior) mode
    model.eval()
    likelihood.eval()

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
    #    test_x = torch.linspace(0, 1, 51)
        observed_pred = likelihood(model(test_x))

    means = observed_pred.mean.cpu()
    MSE = torch.mean((means - test_y.cpu())*(means - test_y.cpu()))
    #print('Test MAE: {}'.format(torch.mean((means - test_y.cpu())*(means - test_y.cpu()))))

    time_ski.append(uTime)
    print('SKI: MSE &Time &RAM &VRAM: {}'.format(MSE), "&", round(uTime,2), "&", round(uRAM,2) , "&" , round(uVRAM,2) )
    likelihood = None
    model = None
    mll = None
    train_x = None
    train_y = None
    torch.cuda.empty_cache()
    gc.collect()

time_ski

# LOVE

# Lanczos Variance Estimates (LOVE)

In [ ]:
time_love = []

for i in np.arange(0,6):
    csvfile = pd.read_csv(data_files_x[i], header = None)
    train_x = torch.tensor(csvfile[0]).float()
    train_x = torch.reshape(train_x,[-1,1])
    csvfile = pd.read_csv(data_files_y[i], header = None)
    train_y = torch.tensor(csvfile[0]).float()
    train_n = int(len(train_x))
    train_x = train_x.contiguous()
    train_y = train_y.contiguous()
    print(train_x.shape)
    print(train_y.shape)
    print(test_x.shape)
    print(test_y.shape)
    if gpu:
        train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()


    my_batch_size = 3200
    smoke_test = False

    from torch.utils.data import TensorDataset, DataLoader
    train_dataset = TensorDataset(train_x, train_y)#batch_size=1024
    train_loader = DataLoader(train_dataset, batch_size=my_batch_size, shuffle=True)

    test_dataset = TensorDataset(test_x, test_y)
    test_loader = DataLoader(test_dataset, batch_size=my_batch_size, shuffle=False)

    class LargeFeatureExtractor(torch.nn.Sequential):
        def __init__(self, input_dim):
            super(LargeFeatureExtractor, self).__init__()
            self.add_module('linear1', torch.nn.Linear(input_dim, 1000))
            self.add_module('relu1', torch.nn.ReLU())
            self.add_module('linear2', torch.nn.Linear(1000, 500))
            self.add_module('relu2', torch.nn.ReLU())
            self.add_module('linear3', torch.nn.Linear(500, 50))
            self.add_module('relu3', torch.nn.ReLU())
            self.add_module('linear4', torch.nn.Linear(50, 1))
            print("VRAM Usage:", torch.cuda.memory_allocated()/(1024**2) , "MB")
            


    class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=1)),
                grid_size=100, num_dims=1,
            )

            # Also add the deep net
            self.feature_extractor = LargeFeatureExtractor(input_dim=train_x.size(-1))

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            # We're also scaling the features so that they're nice values
            projected_x = self.feature_extractor(x)
            projected_x = projected_x - projected_x.min(0)[0]
            projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1

            # The rest of this looks like what we've seen
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            vram_usage()
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


    mem_begin = get_mem()
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPRegressionModel(train_x, train_y, likelihood)

    if gpu:
        model = model.cuda()
        likelihood = likelihood.cuda()


    # 2k, 20k, 200k
    # 30, 
    training_iterations = 30 #


    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


    def train():
        iterator = tqdm.tqdm(range(training_iterations))
        for i in iterator:
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            iterator.set_postfix(loss=loss.item())
            vram_usage()
            optimizer.step()

    #%time train()

    begin = time.time()
    train()
    uTime = time.time()-begin
    uRAM = (get_mem() - mem_begin)/(1024**2)
    uVRAM = max_vram / (1024 ** 2)

    # Get into evaluation (predictive posterior) mode
    model.eval()
    likelihood.eval()

    test_loader = DataLoader(test_dataset, batch_size=320, shuffle=False)
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        if gpu:
            observed_pred = likelihood(model(test_x.to('cuda')))
        else:
            observed_pred = likelihood(model(test_x))

    means = observed_pred.mean.cpu()
    MSE = torch.mean((means - test_y.cpu())*(means - test_y.cpu()))
    #print('Test MAE: {}'.format(torch.mean((means - test_y.cpu())*(means - test_y.cpu()))))
    time_love.append(uTime)
    print('SKI: MSE &Time &RAM &VRAM: {}'.format(MSE), "&", round(uTime,2), "&", round(uRAM,2) , "&" , round(uVRAM,2) )
    likelihood = None
    model = None
    mll = None
    train_x = None
    train_y = None
    torch.cuda.empty_cache()
    gc.collect()

time_ski

# Outputing csv files

In [6]:
with open('time_ski.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    for item in time_ski:
        writer.writerow([item])


with open('time_love.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    for item in time_love:
        writer.writerow([item])